In [1]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
%matplotlib inline

from base import supres, BaseStrategy

# Load Data

In [2]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'

def read_data(pair):
    db = os.path.join(dropbox_dir,'sqlite', 'ploniex-chart-data', f'{pair}.db')
    data = load_trading_data(db, 'chart_data', from_date='2015-01-01', period='30min')
    data['ohlc4'] = (data['open'] + data['close'] + data['high'] + data['low']) / 4
    return data

In [3]:
supports_for_data_dict_test = load_model('supports-for-data-test.pkl')
supports_for_data_dict_train = load_model('supports-for-data.pkl')
supports_for_data_dict = {**supports_for_data_dict_train, **supports_for_data_dict_test}

In [4]:
features = ['support']

In [5]:
train_pairs = pd.read_csv('pairs_train.tsv', header=None)[0].tolist()
test_pairs = pd.read_csv('pairs_test.tsv', header=None)[0].tolist()
all_pairs = pd.read_csv('pairs.tsv', header=None)[0].tolist()
data_dict = {pair: read_data(pair) for pair in all_pairs}

# Parallelize

In [6]:
from multiprocessing import Pool
import time
from tqdm import *

def run_parallel(f, args, n_process=10):
    results = []
    with Pool(processes=n_process) as p:
        all_args = list(args)
        with tqdm_notebook(enumerate(p.imap_unordered(f, all_args)), leave=False, total=len(all_args)) as pbar:
            for i, x in pbar:
                results.append(x)
                pbar.update()
                
        return results

In [7]:
def dict_from_list(l):
    d = {}
    for row in l:
        d_inner = d
        for k in row[:-2]:
            if k not in d_inner.keys():
                d_inner[k] = {}
            d_inner = d_inner[k]
        d_inner[row[-2]] = row[-1]
    return d

# Multicoin strategy

In [19]:
def combine_dicts(d1, d2):
    keys = list(d1.keys()) + list(d2.keys())
    d = {}
    for k in keys:
        v1, v2 = d1.get(k, None), d2.get(k, None)
        if v1 is None:
            d[k] = v2
        elif v2 is None:
            d[k] = v1
        elif type(v1) == dict:
            d[k] = combine_dicts(v1, v2)
        else:
            d[k] = v1 + v2
    return d

In [57]:
class MulticoinStrategy():
    def __init__(self, data_dict, pairs, willing_loss, target_profit, target_return):
        self.data_dict = data_dict
        self.pairs = pairs
        self.willing_loss = willing_loss
        self.target_profit = target_profit
        self.target_return = target_return
        self.strategies = {}
        self.wallet = {'btc': 100}
    
    
    def trade_pair(self, pair):
        contract_pair = ContractPair.new(*[x.lower() for x in pair.split('_')])
        data = self.data_dict[pair]
        dm = OhlcDataManager(data[['open', 'low', 'high', 'close']], data[features])
        wallet = deepcopy(self.wallet)
        wallet[contract_pair['priceContract']] = wallet[contract_pair['priceContract']] / len(self.pairs)
        exchange = BackTestExchange(dm, wallet, 0.0025)
        strategy = BaseStrategy(exchange, 
                                dm, 
                                contract_pair, 
                                willing_loss=self.willing_loss, 
                                target_profit=self.target_profit, 
                                target_return=self.target_return)
        strategy.trade()
        return pair, strategy
    
    def trade(self):
        results = run_parallel(self.trade_pair, self.pairs)
        self.strategies = dict_from_list(results)
                
    
    def wallet_history(self):
        history = {}
        for strategy in self.strategies.values():
            price_c = strategy.contract_pair['priceContract'].upper()
            trade_c = strategy.contract_pair['tradeContract'].upper()
            data = data_dict[f'{price_c}_{trade_c}']
            
            history_s = strategy.exchange.wallet_history
            with_timestamps = {data['timestamp'].iloc[ts] if ts < data.shape[0] else data['timestamp'].iloc[ts-1]: v for ts, v in history_s.items()}
            history = combine_dicts(history, with_timestamps)
            
        return history
    
    
    def all_transactions(self):
        transactions = []
        for strategy in self.strategies.values():
            transactions += strategy.exchange.transactions
        return transactions
    
    def transactions(self, contract_name=None):
        if contract_name is None:
            return self.all_transactions()
        
        return self.strategies[contract_name].exchange.transactions
    
    def stats(self, contract_name):
        return Statistics(contract_name, self.wallet[contract_name])\
            .evaluate(self.all_transactions())

# Setup

In [58]:
window_size = 12
target_profit = 0.01
willing_loss = 0.06
target_return = -0.02

for pair in all_pairs:
    data = data_dict[pair]
    data['support'] = supports_for_data_dict[pair][window_size]

# Train Results

In [60]:
multi_strategy = MulticoinStrategy(data_dict, train_pairs, willing_loss, target_profit, target_return)
multi_strategy.trade()
multi_strategy.stats('btc').report()

,btc
startAmount,100.0000
numberOfTrades,"7,530.0000"
totalProfit,474.4518
avgProfit,0.0630
winPercentage,95.2324
avgWinTrade,0.0987
avgLossTrade,-0.6491
profitFactor,3.0359
maxDrawdown,2.9596
avgTimeToClose,6.5676


In [61]:
history = multi_strategy.wallet_history()

In [62]:


history

{1535481000: {'btc': 43.832904033706775,
  'bat': 101987.71984597771,
  'xmr': 0.0},
 1535486400: {'btc': 15.905550690178924, 'bat': 0.0, 'game': 0.0},
 1535657400: {'btc': 17.81846341018063,
  'bat': 107347.22721762645,
  'pasc': 0.0},
 1535706000: {'btc': 3.3991934873525187, 'bat': 0.0},
 1536193800: {'btc': 75.21390770607269,
  'bat': 0.0,
  'bts': 0.0,
  'dash': 0.0,
  'etc': 0.0,
  'gnt': 447190.2490498924,
  'omg': 0.0},
 1536471000: {'btc': 15.3298464423289,
  'bat': 135752.93566737245,
  'gnt': 463154.5966793633,
  'rep': 8059.095049372343,
  'zec': 0.0},
 1536483600: {'btc': 22.060758904428507, 'bat': 0.0, 'rep': 0.0},
 1536604200: {'btc': 0.060834528658880505, 'bat': 143576.83056429308},
 1536609600: {'btc': 16.41334214395331, 'bat': 0.0, 'game': 0.0},
 1536696000: {'btc': 0.8525538682924849,
  'bat': 148760.7536172574,
  'xmr': 2686.376415919706},
 1536705000: {'btc': 49.056129407476675, 'bat': 0.0, 'xmr': 0.0},
 1536748200: {'btc': 0.06203650008702821, 'bat': 164255.4027314

# Test Results

In [11]:
multi_strategy = MulticoinStrategy(data_dict, test_pairs, willing_loss, target_profit, target_return)
multi_strategy.trade()
multi_strategy.stats('btc').report()

,btc
startAmount,100.0000
numberOfTrades,"7,303.0000"
totalProfit,554.1265
avgProfit,0.0759
winPercentage,95.4676
avgWinTrade,0.1177
avgLossTrade,-0.8052
profitFactor,3.0791
maxDrawdown,4.7714
avgTimeToClose,7.1464
